In [1]:
library(caret)
library(kernlab)
library(foreign)
library(mlbench)
library(rpart)
library(rpart.plot)
library(ipred)
library(randomForest)
library(gbm)
library(nnet)
library(pROC)
library(ggplot2)
library(ROSE)
library(doParallel)
registerDoParallel(6)
getDoParWorkers()

Loading required package: ggplot2

Loading required package: lattice


Attaching package: 'kernlab'


The following object is masked from 'package:ggplot2':

    alpha


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ggplot2':

    margin


Loaded gbm 2.1.8.1

Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var


Loaded ROSE 0.0-4


Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



[1] 6

In [2]:
mydata=read.csv("sepsis data.csv",header=TRUE,sep=",")
dim(mydata)

[1] 555   7

In [3]:
mydata$death=as.factor(mydata$death)
mydata1=mydata[which(mydata$death=="Survival"),]
mydata2=mydata[which(mydata$death=="Death"),]
index1=sample(dim(mydata1)[1],nrow(mydata1)*0.25)
index2=sample(dim(mydata2)[1],nrow(mydata2)*0.25)
testing=rbind(mydata1[index1,],mydata2[index2,])
training=rbind(mydata1[-index1,],mydata2[-index2,])

In [4]:
fitControl=trainControl(method="repeatedcv",
                        number=3,
                        repeats=2,
                        classProbs=TRUE,   
                        summaryFunction=twoClassSummary,
                        search="random")

In [5]:
model_caret=caret::modelLookup()
head(model_caret)
dim(model_caret)

,model,parameter,label,forReg,forClass,probModel
,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>
1,ada,iter,#Trees,FALSE,TRUE,TRUE
2,ada,maxdepth,Max Tree Depth,FALSE,TRUE,TRUE
3,ada,nu,Learning Rate,FALSE,TRUE,TRUE
4,AdaBag,mfinal,#Trees,FALSE,TRUE,TRUE
5,AdaBag,maxdepth,Max Tree Depth,FALSE,TRUE,TRUE
9,adaboost,nIter,#Trees,FALSE,TRUE,TRUE


[1] 511   6

In [6]:
model_caret_forclass=model_caret[which(model_caret$forClass=="TRUE"),]
head(model_caret_forclass)
nrow(model_caret_forclass)

,model,parameter,label,forReg,forClass,probModel
,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>
1,ada,iter,#Trees,FALSE,TRUE,TRUE
2,ada,maxdepth,Max Tree Depth,FALSE,TRUE,TRUE
3,ada,nu,Learning Rate,FALSE,TRUE,TRUE
4,AdaBag,mfinal,#Trees,FALSE,TRUE,TRUE
5,AdaBag,maxdepth,Max Tree Depth,FALSE,TRUE,TRUE
9,adaboost,nIter,#Trees,FALSE,TRUE,TRUE


[1] 431

In [7]:
model_caret_forreg=model_caret[which(model_caret$forReg=="TRUE"),]
head(model_caret_forreg)
nrow(model_caret_forreg)

,model,parameter,label,forReg,forClass,probModel
,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>
12,ANFIS,num.labels,#Fuzzy Terms,TRUE,FALSE,FALSE
13,ANFIS,max.iter,Max. Iterations,TRUE,FALSE,FALSE
14,avNNet,size,#Hidden Units,TRUE,TRUE,TRUE
15,avNNet,decay,Weight Decay,TRUE,TRUE,TRUE
16,avNNet,bag,Bagging,TRUE,TRUE,TRUE
20,bag,vars,#Randomly Selected Predictors,TRUE,TRUE,TRUE


[1] 296

In [8]:
modellist=model_caret_forclass$model[1:100]

In [9]:
M=length(modellist)
M
AUC_train=array(0,M)
AUC_test=array(0,M)

[1] 100

In [ ]:
for (i in 1:length(modellist)){
    skip_to_next <- FALSE
    tryCatch(modelfit<-train(death~.,
                             data=training,
                             method=modellist[i],
                             preProc=c("center","scale"),
                             metric="ROC",
                             tuneLength=10,
                             trControl=fitControl),error=function(e){skip_to_next <<- TRUE})
    if(skip_to_next) { next }      
    predictions_train=predict(modelfit,newdata=training)
    predictions_test=predict(modelfit,newdata=testing)
    predictions_train
    predictions_test
    confusionMatrix(predict(modelfit,training),training$death)
    confusionMatrix(predict(modelfit,testing),testing$death)
    train_results=predict(modelfit,training,type="prob")
    test_results=predict(modelfit,testing,type="prob")
    train_results$obs=training$death
    train_results$pred=predictions_train
    test_results$obs=testing$death
    test_results$pred=predictions_test
    ROC_train<-roc(training$death,train_results[,"Death"],levels=c("Survival","Death"))
    ROC_test<-roc(testing$death,test_results[,"Death"],levels=c("Survival","Death"))
    AUC_train[i]=ROC_train[9]$auc
    AUC_test[i]=ROC_test[9]$auc
    cat('Completed running Model', i, '(',modellist[i],')','of', M,'\n')
  }

Setting direction: controls < cases

Setting direction: controls < cases



Completed running Model 1 ( ada ) of 100 


Setting direction: controls < cases

Setting direction: controls < cases



Completed running Model 2 ( ada ) of 100 


In [12]:
for (i in 1:length(modellist)){
    tryCatch(modelfit<-train(death~.,
                             data=training,
                             method=modellist[i],
                             preProc=c("center","scale"),
                             metric="ROC",
                             tuneLength=10,
                             trControl=fitControl),error=function(e){print('Error')})
    }

[1] "Error"
[1] "Error"
[1] "Error"


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."


Something is wrong; all the ROC metric values are missing:
      ROC           Sens          Spec    
 Min.   : NA   Min.   : NA   Min.   : NA  
 1st Qu.: NA   1st Qu.: NA   1st Qu.: NA  
 Median : NA   Median : NA   Median : NA  
 Mean   :NaN   Mean   :NaN   Mean   :NaN  
 3rd Qu.: NA   3rd Qu.: NA   3rd Qu.: NA  
 Max.   : NA   Max.   : NA   Max.   : NA  
 NA's   :10    NA's   :10    NA's   :10   
[1] "Error"


Loading required namespace: kohonen

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."


Something is wrong; all the ROC metric values are missing:
      ROC           Sens          Spec    
 Min.   : NA   Min.   : NA   Min.   : NA  
 1st Qu.: NA   1st Qu.: NA   1st Qu.: NA  
 Median : NA   Median : NA   Median : NA  
 Mean   :NaN   Mean   :NaN   Mean   :NaN  
 3rd Qu.: NA   3rd Qu.: NA   3rd Qu.: NA  
 Max.   : NA   Max.   : NA   Max.   : NA  
 NA's   :6     NA's   :6     NA's   :6    
[1] "Error"


...

[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] 

Warning message in train.default(x, y, weights = w, ...):
"Class probabilities were requested for a model that does not implement them"


[1] "Error"


Warning message in train.default(x, y, weights = w, ...):
"Class probabilities were requested for a model that does not implement them"


[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] "Error"
[1] 

In [10]:
for (i in 1:length(modellist)){
    tryCatch(modelfit<-train(death~.,
                             data=training,
                             method=modellist[i],
                             preProc=c("center","scale"),
                             metric="ROC",
                             tuneLength=10,
                             trControl=fitControl),error=function(e){print('Error')})
    predictions_train=predict(modelfit,newdata=training)
    predictions_test=predict(modelfit,newdata=testing)
    predictions_train
    predictions_test
    confusionMatrix(predict(modelfit,training),training$death)
    confusionMatrix(predict(modelfit,testing),testing$death)
    train_results=predict(modelfit,training,type="prob")
    test_results=predict(modelfit,testing,type="prob")
    train_results$obs=training$death
    train_results$pred=predictions_train
    test_results$obs=testing$death
    test_results$pred=predictions_test
    ROC_train<-roc(training$death,train_results[,"Death"],levels=c("Survival","Death"))
    ROC_test<-roc(testing$death,test_results[,"Death"],levels=c("Survival","Death"))
    AUC_train[i]=ROC_train[9]$auc
    AUC_test[i]=ROC_test[9]$auc
    cat('Completed running Model', i, '(',modellist[i],')','of', M,'\n')
  }

[1] "Error"


ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'object' in selecting a method for function 'predict': object 'modelfit' not found


In [ ]:
AUC=cbind(AUC_train,AUC_test)
AUC=as.data.frame(AUC)
AUC$rowname<-modellist
print(AUC)